In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image as keras_image
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

    Load and preprocess an image from the given path.
    Parameters:
        image_path (str): Path to the image file.
        target_size (tuple): Target size for resizing the image.
    Returns:
        np.array: Preprocessed image array.

In [ ]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = keras.applications.vgg19.preprocess_input(img_array)
    return img_array

    Compute content loss between the content image and the generated image.
    Parameters:
        content (tf.Tensor): Feature representation of the content image.
        combination (tf.Tensor): Feature representation of the generated image.
    Returns:
        tf.Tensor: Content loss.

In [ ]:
# Function to compute content loss
def content_loss(content, combination):
    return tf.reduce_sum(tf.square(combination - content))

    Compute style loss between the style image and the generated image.
    Parameters:
        style (tf.Tensor): Feature representation of the style image.
        combination (tf.Tensor): Feature representation of the generated image.
    Returns:
        tf.Tensor: Style loss.

In [ ]:
# Function to compute style loss
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = 256 * 256
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

    Compute the Gram matrix for a given tensor.
    Parameters:
        x (tf.Tensor): Input tensor.
    Returns:
        tf.Tensor: Gram matrix.

In [ ]:
# Function to compute gram matrix
def gram_matrix(x):
    features = tf.keras.backend.batch_flatten(tf.keras.backend.permute_dimensions(x, (2, 0, 1)))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

    Compute total variation loss to reduce noise in the generated image.
    Parameters:
        x (tf.Tensor): Input tensor.
    Returns:
        tf.Tensor: Total variation loss.

In [ ]:
# Function to compute total variation loss (to reduce noise)
def total_variation_loss(x):
    a = tf.square(x[:, :255, :255, :] - x[:, 1:, :255, :])
    b = tf.square(x[:, :255, :255, :] - x[:, :255, 1:, :])
    return tf.reduce_sum(tf.pow(a + b, 1.25))

    Build the style transfer model using the VGG19 architecture.
    Returns:
        tf.keras.Model: Style transfer model.

In [ ]:
# Function to build the style transfer model
def build_style_transfer_model():
    vgg19 = VGG19(weights='imagenet', include_top=False)

    # Get intermediate layers for content and style representations
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    content_outputs = [vgg19.get_layer(name).output for name in content_layers]
    style_outputs = [vgg19.get_layer(name).output for name in style_layers]

    model_outputs = content_outputs + style_outputs

    # Build model
    model = keras.models.Model(inputs=vgg19.input, outputs=model_outputs)
    model.trainable = False

    return model

    Compute the total style transfer loss.
    Parameters:
        content_image (tf.Tensor): Content image.
        style_image (tf.Tensor): Style image.
        generated_image (tf.Tensor): Generated image.
        alpha (float): Weight for total variation loss.
        beta (float): Weight for the total style transfer loss.
    Returns:
        tf.Tensor: Total style transfer loss.

In [ ]:
# Function to compute style transfer loss
def style_transfer_loss(content_image, style_image, generated_image, alpha=1e4, beta=1e-4):
    content_features = model(content_image)[:len(content_layers)]
    style_features = model(style_image)[len(content_layers):]
    generated_features = model(generated_image)

    loss = 0.0

    # Content loss
    for content, generated in zip(content_features, generated_features[:len(content_layers)]):
        loss += content_loss(content, generated)

    # Style loss
    for style, generated in zip(style_features, generated_features[len(content_layers):]):
        loss += style_loss(style, generated)

    # Total Variation loss (to reduce noise)
    loss += alpha * tf.image.total_variation(generated_image)

    return beta * loss

    Download and load the WikiArt and MS COCO datasets using TensorFlow Datasets.
    Returns:
        tuple: A tuple containing content images and style images.

In [ ]:
# Function to download and load datasets
def download_and_load_datasets():
    # Download WikiArt dataset
    wikiart_builder = tfds.builder('wikiart')
    wikiart_builder.download_and_prepare()
    wikiart = wikiart_builder.as_dataset(split='train[:10%]')  # Adjust the split as needed
    wikiart_images = [img['image'] for img in tfds.as_numpy(wikiart)]

    # Download MS COCO dataset
    coco_builder = tfds.builder('coco/2017')
    coco_builder.download_and_prepare()
    coco = coco_builder.as_dataset(split='train[:10%]')  # Adjust the split as needed
    coco_images = [img['image'] for img in tfds.as_numpy(coco)]
    
    return coco_images, wikiart_images

    Display content, style, and generated images side by side.
    Parameters:
        content (tf.Tensor): Content image.
        style (tf.Tensor): Style image.
        generated (tf.Tensor): Generated image.

In [ ]:
# Function to display images
def display_images(content, style, generated):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(content[0]))
    plt.title('Content Image')

    plt.subplot(1, 3, 2)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(style[0]))
    plt.title('Style Image')

    plt.subplot(1, 3, 3)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(generated[0]))
    plt.title('Generated Image')

    plt.show()

In [ ]:
# Download datasets
content_images, style_images = download_and_load_datasets()

In [ ]:
# Split datasets
train_content_images, val_content_images = train_test_split(content_images, test_size=0.2, random_state=42)
train_style_images, val_style_images = train_test_split(style_images, test_size=0.2, random_state=42)

In [ ]:
# Build the style transfer model
model = build_style_transfer_model()

In [ ]:
# Define the optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.001)

In [ ]:
# Training loop with visualization
num_epochs = 10
batch_size = 16

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Iterate over batches
    for i in range(0, len(train_content_images), batch_size):
        batch_content = train_content_images[i:i + batch_size]
        batch_style = train_style_images[i:i + batch_size]

        with tf.GradientTape() as tape:
            loss = style_transfer_loss(batch_content, batch_style, model(batch_content))
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Display visual examples during training
        if i % 500 == 0:
            display_images(batch_content, batch_style, model(batch_content))

    # Validate the model on the validation set
    val_loss = style_transfer_loss(val_content_images, val_style_images, model(val_content_images)).numpy()
    print(f"Validation Loss: {val_loss}")

In [ ]:
# Save the trained model
model.save('style_transfer_model.h5')

In [ ]:
# Download and load COCO 2017 test dataset for testing the model using the Tensorflow Dataset
coco_test_builder = tfds.builder('coco/2017')
coco_test_builder.download_and_prepare()
coco_test = coco_test_builder.as_dataset(split='test')
coco_test_images = [img['image'] for img in tfds.as_numpy(coco_test.take(1))]

In [ ]:
# Take input for the test image path from the user
test_image_path = input("Enter the path to the test image: ")

In [ ]:
# Load and preprocess the user-provided test image
test_image = load_and_preprocess_image(test_image_path)

In [ ]:
# Visualize style transfer on the test image
generated_image = model(test_image)

In [ ]:
# Display the test image and the generated image
display_images(test_image, np.zeros_like(test_image), generated_image)

    Limitations:
        1) Single Style Transfer
        2) Parameter Sensitivity
        3) Limited Image Resolution
        4) Content and Style Dimension Mismatch
    Improvements:
        1) Architectural Enhancements
        2) Algorithmic Improvements
        3) Using a Diverse Dataset